In [1]:
%load_ext sparksql_magic
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

spark = SparkSession.builder.appName("dim_repo").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 18:44:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/11 18:44:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.json("/data/*.json.gz")
# Filter for IssuesEvent
df = df.filter(df["type"] == "IssuesEvent")
df = df.select(
    col("actor"),
    col("created_at"),
    col("id").alias("event_id"),
    col("org"),
    col("repo"),
    col("payload.action"),
    col("payload.issue")
)

25/03/11 18:46:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df.createOrReplaceTempView("df")

## For github repo dim data


In [7]:
%%sql
MERGE INTO iceberg.dim_repo AS target USING (
  SELECT DISTINCT
    CAST(repo.id AS BIGINT) AS id,
    repo.name,
    repo.url
  FROM
    df
) AS source ON target.id = source.id WHEN MATCHED THEN
UPDATE
SET
  target.name = source.name,
  target.url = source.url WHEN NOT MATCHED THEN INSERT *;

++
||
++
++

In [15]:
%%sql 
SELECT
  *
FROM
  iceberg.dim_repo

id,name,url
716240168,AshwinAnand868/car-rental-app,https://api.github.com/repos/AshwinAnand868/car-rental-app
291473513,VolkovLabs/volkovlabs-dynamictext-panel,https://api.github.com/repos/VolkovLabs/volkovlabs-dynamictext-panel
704981549,rainflame/pika-routes,https://api.github.com/repos/rainflame/pika-routes
103769370,rvlenth/emmeans,https://api.github.com/repos/rvlenth/emmeans
582137397,ASJordi/website-activity-status,https://api.github.com/repos/ASJordi/website-activity-status
67305738,dtolnay/syn,https://api.github.com/repos/dtolnay/syn
713640829,eryjus/16bcfs,https://api.github.com/repos/eryjus/16bcfs
619931831,wlky-0106/upptime,https://api.github.com/repos/wlky-0106/upptime
35810174,electron-react-boilerplate/electron-react-boilerplate,https://api.github.com/repos/electron-react-boilerplate/electron-react-boilerplate
736471558,DanielKim7305/first_repo,https://api.github.com/repos/DanielKim7305/first_repo
